In [2]:
!pip install mapboxgl

  Using cached https://files.pythonhosted.org/packages/d3/1b/9f79b01afc38644ea13ef30d9836a1211cd35f9a4d50403f79a1b187876f/mapboxgl-0.10.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/74/46/e81907704ab203206769dee1385dc77e1407576ff8f50a0681d0a6b541be/colour-0.1.5-py2.py3-none-any.whl


In [3]:
# set up work environment
%matplotlib inline
import pandas as pd, numpy as np, matplotlib.pyplot as plt
import geopandas as gpd
from geopandas import GeoDataFrame
from shapely.geometry import Point
from scipy import ndimage
import matplotlib.pylab as pylab
import json
import requests
import pprint
import numpy as np
from mapboxgl.viz import *
from mapboxgl.utils import *
import ipywidgets as widgets

pylab.rcParams['figure.figsize'] = 10, 8

import warnings 
warnings.filterwarnings('ignore')

In [4]:
# mapbox token
token = 'pk.eyJ1IjoicmFjaGVsb20iLCJhIjoiY2puYXE2YnV4NTVmcjNxbjE2ZHdjazVlbCJ9.jZKiF7-Y9c1gIGDvO0SmwA'

### Map: Household Poverty Rate (By Census Tract)

In [5]:
# Select the year:
pick_year = widgets.Dropdown(options=['Select Year', '2016', '2015', '2014'],value='Select Year',description='Year:',disabled=False)
display(pick_year)

Dropdown(description='Year:', options=('Select Year', '2016', '2015', '2014'), value='Select Year')

In [6]:
# Select the state
pick_state = widgets.Dropdown(options=['Select State', 'CA', 'OR', 'WA'],value='Select State',description='State:',disabled=False)
display(pick_state)

Dropdown(description='State:', options=('Select State', 'CA', 'OR', 'WA'), value='Select State')

In [7]:
dictionary = {
    "CA" : '06',
    "OR" : '41',
    "WA" : '53'
}

for st, code in dictionary.items():
    if st == pick_state.value:
        selected_state = code

In [8]:
selected_state

'06'

In [9]:
# visualize poverty rate by census tract
# set up the api
endpoint = 'https://api.census.gov/data/'
# the year is set by the drop down menu above
year = pick_year.value
# can replace with another dataset: https://api.census.gov/data.html
dataset = 'acs/acs5'
# can replace with other variables: https://api.census.gov/data/2016/acs/acs5/variables.html
tables = 'B17001_001E,B17001_002E'
# the state is set by the drop down menu above
state = selected_state
# can replace with other county codes: https://www.census.gov/geo/reference/codes/cou.html
county = '037'
# can replace with another key: https://api.census.gov/data/key_signup.html
key = '743d7a3fd3ab38085c25887ef6bed9a331092929'

# this query gets data for the specified year, dataset, and tables for ALL census tracts in the specified state and county
url = requests.Request('GET', endpoint+year+'/'+dataset+'?get=NAME,'+tables+\
                       '&for=tract:*&in=state:'+state+'&in=county:'+county+'&key='+key).prepare().url

print(url)

https://api.census.gov/data/2016/acs/acs5?get=NAME,B17001_001E,B17001_002E&for=tract:*&in=state:06&in=county:037&key=743d7a3fd3ab38085c25887ef6bed9a331092929


In [10]:
response = requests.get(url)
results = response.text
# view first 500 characters of query to make sure data is correct
print(results[:500])

[["NAME","B17001_001E","B17001_002E","state","county","tract"],
["Census Tract 1011.10, Los Angeles County, California","4557","632","06","037","101110"],
["Census Tract 1011.22, Los Angeles County, California","3293","135","06","037","101122"],
["Census Tract 1012.10, Los Angeles County, California","5791","1558","06","037","101210"],
["Census Tract 1012.20, Los Angeles County, California","3167","476","06","037","101220"],
["Census Tract 1013, Los Angeles County, California","4191","333","06",


In [11]:
data = json.loads(results)
# convert data to dataframe so we can analyze

# create list of columns to use
columns = ["Description", "TotalPop","In_Poverty","state", "county", "tract"]

# create dataframe from data and drop first row
poverty =pd.DataFrame.from_records(data,columns=columns).drop(0)
# view new data frame
poverty.head()

,Description,TotalPop,In_Poverty,state,county,tract
1,"Census Tract 1011.10, Los Angeles County, Cali...",4557,632,06,037,101110
2,"Census Tract 1011.22, Los Angeles County, Cali...",3293,135,06,037,101122
3,"Census Tract 1012.10, Los Angeles County, Cali...",5791,1558,06,037,101210
4,"Census Tract 1012.20, Los Angeles County, Cali...",3167,476,06,037,101220
5,"Census Tract 1013, Los Angeles County, California",4191,333,06,037,101300


In [12]:
# convert numeric columns from string to integer
poverty[["TotalPop","In_Poverty","tract"]] = poverty[["TotalPop","In_Poverty","tract"]].astype(int)

# add new column for percent of individuals in poverty
poverty["Poverty Rate"] = round((poverty["In_Poverty"]/poverty["TotalPop"]*100),2)

poverty.head()

,Description,TotalPop,In_Poverty,state,county,tract,Poverty Rate
1,"Census Tract 1011.10, Los Angeles County, Cali...",4557,632,06,037,101110,13.87
2,"Census Tract 1011.22, Los Angeles County, Cali...",3293,135,06,037,101122,4.10
3,"Census Tract 1012.10, Los Angeles County, Cali...",5791,1558,06,037,101210,26.90
4,"Census Tract 1012.20, Los Angeles County, Cali...",3167,476,06,037,101220,15.03
5,"Census Tract 1013, Los Angeles County, California",4191,333,06,037,101300,7.95


In [13]:
# bring in los angeles county census tracts
la_tracts = gpd.read_file('data/LA_tracts.geojson')
la_tracts.head()

,shape_area,label,x_center,ct10,y_center,geoid10,shape_len,geometry
0,4025735684.42,9110.01,6620403.0,911001,1998891.0,06037911001,353933.808192,(POLYGON ((-117.6671211134298 34.5580081383781...
1,2078689856.02,9800.03,6575300.0,980003,2112006.0,06037980003,273188.86321,(POLYGON ((-117.8806120004425 34.7636159996687...
2,11118018325.1,9303.01,6603027.0,930301,1932124.0,06037930301,628603.531323,(POLYGON ((-117.6552358388393 34.3972219664572...
3,4824001.88224,5730.03,6500215.0,573003,1747305.0,06037573003,9050.00845755,(POLYGON ((-118.1992330000361 33.7971229999225...
4,6697030.7108,2976.02,6473372.0,297602,1719119.0,06037297602,12308.3153848,(POLYGON ((-118.2879799997457 33.7225829999421...


In [14]:
la_tracts = la_tracts.drop(labels=['shape_area','label','x_center','y_center','shape_len'], axis=1)
la_tracts = la_tracts.rename({'ct10':'tract'}, axis=1)
la_tracts.head()

,tract,geoid10,geometry
0,911001,06037911001,(POLYGON ((-117.6671211134298 34.5580081383781...
1,980003,06037980003,(POLYGON ((-117.8806120004425 34.7636159996687...
2,930301,06037930301,(POLYGON ((-117.6552358388393 34.3972219664572...
3,573003,06037573003,(POLYGON ((-118.1992330000361 33.7971229999225...
4,297602,06037297602,(POLYGON ((-118.2879799997457 33.7225829999421...


In [15]:
la_tracts['tract'] = la_tracts['tract'].astype(int)

In [16]:
la_poverty = la_tracts.merge(poverty, on='tract')
la_poverty.head()

,tract,geoid10,geometry,Description,TotalPop,In_Poverty,state,county,Poverty Rate
0,911001,06037911001,(POLYGON ((-117.6671211134298 34.5580081383781...,"Census Tract 9110.01, Los Angeles County, Cali...",4066,555,06,037,13.65
1,980003,06037980003,(POLYGON ((-117.8806120004425 34.7636159996687...,"Census Tract 9800.03, Los Angeles County, Cali...",0,0,06,037,NaN
2,930301,06037930301,(POLYGON ((-117.6552358388393 34.3972219664572...,"Census Tract 9303.01, Los Angeles County, Cali...",551,122,06,037,22.14
3,573003,06037573003,(POLYGON ((-118.1992330000361 33.7971229999225...,"Census Tract 5730.03, Los Angeles County, Cali...",1931,631,06,037,32.68
4,297602,06037297602,(POLYGON ((-118.2879799997457 33.7225829999421...,"Census Tract 2976.02, Los Angeles County, Cali...",3929,541,06,037,13.77


In [17]:
la_poverty = la_poverty.dropna(subset=['Poverty Rate'])
la_poverty.head()

,tract,geoid10,geometry,Description,TotalPop,In_Poverty,state,county,Poverty Rate
0,911001,06037911001,(POLYGON ((-117.6671211134298 34.5580081383781...,"Census Tract 9110.01, Los Angeles County, Cali...",4066,555,06,037,13.65
2,930301,06037930301,(POLYGON ((-117.6552358388393 34.3972219664572...,"Census Tract 9303.01, Los Angeles County, Cali...",551,122,06,037,22.14
3,573003,06037573003,(POLYGON ((-118.1992330000361 33.7971229999225...,"Census Tract 5730.03, Los Angeles County, Cali...",1931,631,06,037,32.68
4,297602,06037297602,(POLYGON ((-118.2879799997457 33.7225829999421...,"Census Tract 2976.02, Los Angeles County, Cali...",3929,541,06,037,13.77
5,576301,06037576301,(POLYGON ((-118.1850160000802 33.7825970000275...,"Census Tract 5763.01, Los Angeles County, Cali...",4134,1200,06,037,29.03


In [18]:
la_poverty = la_poverty.drop(labels=['state','county','Description','TotalPop','In_Poverty'], axis=1)
la_poverty = la_poverty.rename({'tract':'Cenus Tract','geoid10':'FIPS Code', 'geometry':'geometry', 'Zero_Vehicle_Percent':'Percent of Households with No Vehicle'},axis=1)
la_poverty.head()

,Cenus Tract,FIPS Code,geometry,Poverty Rate
0,911001,06037911001,(POLYGON ((-117.6671211134298 34.5580081383781...,13.65
2,930301,06037930301,(POLYGON ((-117.6552358388393 34.3972219664572...,22.14
3,573003,06037573003,(POLYGON ((-118.1992330000361 33.7971229999225...,32.68
4,297602,06037297602,(POLYGON ((-118.2879799997457 33.7225829999421...,13.77
5,576301,06037576301,(POLYGON ((-118.1850160000802 33.7825970000275...,29.03


In [19]:
la_poverty.describe()

,Cenus Tract,Poverty Rate
count,2317.000000,2317.000000
mean,402286.995684,18.211001
std,222277.454427,12.217563
min,101110.000000,0.000000
25%,211122.000000,8.670000
50%,404302.000000,15.530000
75%,551202.000000,25.590000
max,980031.000000,88.470000


In [20]:
la_poverty.to_file('data/la_poverty.geojson', driver='GeoJSON')

In [21]:
# this is for reading the data within datahub
viz = ChoroplethViz('data/la_poverty.geojson', 
                    access_token=token,
                    color_property='Poverty Rate',
                    color_stops=create_color_stops([0, 8.5, 15.5, 25.5,88.5,100], colors='YlOrRd'),
                    color_function_type='interpolate',
                    line_stroke='-',
                    line_color='grey',
                    line_width=0.25,
                    opacity=0.8,
                    center=(-118.265986,34.036506),
                    zoom=8,
                    below_layer='waterway-label',
                    legend_layout='horizontal',
                    legend_key_shape='bar',
                    legend_key_borders_on=False)
viz.show()

In [22]:
# this is for online viewing -- uploaded to GitHub
viz_online = ChoroplethViz('https://raw.githubusercontent.com/omrachel/equitymetrics/master/la_poverty.geojson', 
                    access_token=token,
                    color_property='Poverty Rate',
                    color_stops=create_color_stops([0, 8.5, 15.5, 25.5,88.5,100], colors='YlOrRd'),
                    color_function_type='interpolate',
                    line_stroke='-',
                    line_color='grey',
                    line_width=0.25,
                    opacity=0.8,
                    center=(-118.265986,34.036506),
                    zoom=8,
                    below_layer='waterway-label',
                    legend_layout='horizontal',
                    legend_key_shape='bar',
                    legend_key_borders_on=False)
viz_online.show()

In [23]:
with open('viz_online_poverty.html', 'w') as f:
    f.write(viz_online.create_html())